In [1]:
import pandas as pd
import numpy as np
import re
import networkx as nx
import utils.helpers as hpr
import ast
import collections
import requests as rq
from bs4 import BeautifulSoup
import re

### Get project team deliverables

In [66]:
request_teams = rq.get("https://governance.openstack.org/tc/reference/projects/index.html")
soup_teams = BeautifulSoup(request_teams.text, 'html.parser')

teams_deliverables = {}
href_links_teams = soup_teams.find_all("a", "reference internal")

for i in range(1, len(href_links_teams)-2):
    link = href_links_teams[i]
    link_suffix = link["href"]

    request_projects = rq.get("https://governance.openstack.org/tc/reference/projects/%s" % (link_suffix))
    soup_projects = BeautifulSoup(request_projects.text, 'html.parser')
    href_links_projects = soup_projects.find_all("a", "reference internal")

    projects = [href_links_projects[j].text for j in range(3, len(href_links_projects))]
    teams_deliverables[link_suffix[:-5].replace("-", "")] = projects

In [35]:
pd.DataFrame({"team": teams_deliverables.keys(), "projects": teams_deliverables.values()}).to_csv("./RQs/PQ/Files/teams_deleverables.csv", index=False)

### Associate projects belongng to more than 2 services

In [3]:
mapped_projects = pd.read_csv("./all_os_components.csv")
mapped_projects["related_projects"] = mapped_projects["related_projects"].map(ast.literal_eval)

In [68]:
projects = hpr.flatten_list(mapped_projects["related_projects"].values)
projects_count = collections.Counter(projects)
df_projects = pd.DataFrame({"k": projects_count.keys(), "v": projects_count.values()})
common_projects = df_projects.loc[df_projects["v"] >= 2, "k"].values

In [69]:
len(projects_count)-len(df_projects.loc[df_projects["v"] == 1, "k"].values)

93

In [70]:
def get_affiliated_services(p):
    result = []
    for idx, row in mapped_projects.iterrows():
        if p in row["related_projects"]:
            result.append(row["main_project"])
    return result

In [71]:
result = {}
for p in common_projects:
    result[p] = get_affiliated_services(p)

In [72]:
df_projects_mapping = pd.DataFrame({"project": result.keys(), "teams": result.values()})
df_projects_mapping["count"] = df_projects_mapping["teams"].map(lambda x: len(x))
df_projects_mapping.sort_values(by="count", ascending=0).to_csv("./projects_mapping.csv", index=False)

In [73]:
inv_team_deliverables = {v2: k for k, v in teams_deliverables.items() for v2 in v }

In [74]:
def remove_from_service(df, project, services):
    for s in services:
        for idx,row in df.iterrows():
            if row["main_project"] == s:
                old_list = row["related_projects"]
                old_list.remove(project)
                df._set_value(idx,'related_projects', old_list)

def add_to_service(df, project, service):
    for idx,row in df.iterrows():
        if row["main_project"] == service:
            old_list = row["related_projects"]
            old_list.append(project)
            df._set_value(idx,'related_projects',old_list)
            break

In [75]:
len(result)

93

In [76]:
found_projects_services = []
keys = inv_team_deliverables.keys()
for p,services in result.items():
    if p in keys:
        # Remove k from all associated services
        remove_from_service(mapped_projects, p, services)
        # Add k to inv_team_deliverables.get(k)
        add_to_service(mapped_projects, p, inv_team_deliverables.get(p))
        found_projects_services.append(p)

In [77]:
def remove_remaining_projects(df, projects):
    for idx,row in df.iterrows():
        for item in projects:
            if item in row["related_projects"]:
                old_list = row["related_projects"]
                old_list.remove(item)
                df._set_value(idx,'related_projects', old_list)

In [78]:
left_projects = collections.Counter(hpr.flatten_list(mapped_projects["related_projects"].values))
left_projects = pd.DataFrame({"k": left_projects.keys(), "v": left_projects.values()})

In [79]:
projects_to_remove = left_projects.loc[left_projects["v"] > 1, "k"].values
print("Before Association", len(projects_to_remove))

Before Association 82


In [80]:
projects_to_remove

array(['loci-nova', 'novajoin-tempest-plugin',
       'ansible-role-tripleo-nova', 'openstack-ansible-os_zun',
       'loci-ironic', 'ansible-role-tripleo-ironic',
       'ironic-inspector-tempest-plugin',
       'openstack-ansible-os_swift_sync', 'loci-swift',
       'ansible-role-tripleo-swift', 'loci-cinder',
       'ansible-role-tripleo-cinder', 'ansible-role-k8s-cinder',
       'openstack-ansible-os_manila', 'ansible-role-tripleo-manila',
       'ansible-role-tripleo-neutron', 'loci-neutron', 'neutron-lbaas',
       'ansible-role-tripleo-octavia', 'ansible-role-tripleo-designate',
       'loci-designate', 'ansible-role-k8s-keystone',
       'ansible-role-tripleo-keystone', 'loci-keystone',
       'openstack-ansible-os_placement', 'loci-glance',
       'ansible-role-k8s-glance', 'ansible-role-tripleo-glance',
       'ansible-role-tripleo-barbican', 'ansible-role-tripleo-heat',
       'loci-heat', 'openstack-ansible-os_senlin',
       'openstack-ansible-os_mistral', 'ansible-role-tr

In [81]:
remove_remaining_projects(mapped_projects, projects_to_remove)

In [82]:
left_projects2 = collections.Counter(hpr.flatten_list(mapped_projects["related_projects"].values))
left_projects2 = pd.DataFrame({"k": left_projects2.keys(), "v": left_projects2.values()})

In [83]:
print("After Association", len(left_projects2.loc[left_projects2["v"] > 1, "k"].values))

After Association 0


In [84]:
len(hpr.flatten_list(mapped_projects["related_projects"].values))

849

In [85]:
mapped_projects["length"] = mapped_projects["related_projects"].map(lambda x: len(x)) 

In [86]:
len(mapped_projects[mapped_projects["length"]==0])

0

In [87]:
non_associated_projects = pd.read_csv("./non_associated_projects.csv")
non_associated_projects = non_associated_projects["project"].values.tolist()

In [88]:
already_associated_projects = hpr.flatten_list(mapped_projects["related_projects"].values)
associated_github_projects = []
for p in non_associated_projects:
    if (p in keys) and (p not in already_associated_projects):
        add_to_service(mapped_projects, p, inv_team_deliverables.get(p))
        associated_github_projects.append(p)
        # print(p, inv_team_deliverables.get(p))

In [89]:
mapped_projects = mapped_projects.loc[mapped_projects["length"] != 0]

In [90]:
result = []
mapped_projects_list = hpr.flatten_list(mapped_projects["related_projects"].values)
for p in associated_github_projects:
    if p not in mapped_projects_list:
        result.append(p)

In [91]:
len(associated_github_projects)

114

In [92]:
len(hpr.flatten_list(mapped_projects["related_projects"].values))

963

In [93]:
def remove_projects_are_services(df):
    services = df["main_project"].values.tolist()
    projects_services = []
    for idx,row in df.iterrows():
        projects_services.append(list(set(row["related_projects"]).intersection(services)))
        new_projects = list(set(row["related_projects"]).difference(services))
        df._set_value(idx,'related_projects', new_projects)
    projects_services = hpr.flatten_list(projects_services)
    for idx,row in df.iterrows():
        if row["main_project"] in projects_services:
            new_list = list(dict.fromkeys(row["related_projects"] + [row["main_project"]]))
            df._set_value(idx,'related_projects', new_list)

In [94]:
remove_projects_are_services(mapped_projects)

In [122]:
mapped_projects[["main_project", "related_projects"]].to_csv("./all_os_components.csv", index=False)

### New appraoch for linking projects to their respetive services

In [ ]:
request_teams = rq.get("https://governance.openstack.org/tc/reference/projects/index.html")
soup_teams = BeautifulSoup(request_teams.text, 'html.parser')

teams_deliverables = {}
href_links_teams = soup_teams.find_all("a", "reference internal")

for i in range(1, len(href_links_teams)-2):
    link = href_links_teams[i]
    link_suffix = link["href"]

    request_projects = rq.get("https://governance.openstack.org/tc/reference/projects/%s" % (link_suffix))
    soup_projects = BeautifulSoup(request_projects.text, 'html.parser')
    href_links_projects = soup_projects.find_all("a", "reference internal")

    projects = [href_links_projects[j].text for j in range(3, len(href_links_projects))]
    teams_deliverables[link_suffix[:-5].replace("-", "")] = projects